In [24]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Data Preprocessing

# Reshape the images to the format (num_samples, height, width, num_channels)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Converting the data type to float32 and scale pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# One-hot encoding
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [25]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [26]:
from skimage.transform import resize

# Resize the MNIST images to (224, 224) and replicate the single channel to three channels
x_train = np.array([resize(x, (32, 32, 3)) for x in x_train])
x_val = np.array([resize(x, (32, 32, 3)) for x in x_val])
x_test = np.array([resize(x, (32, 32, 3)) for x in x_test])

In [27]:
# Define ResNet-50 Model
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D

# Load the ResNet-50 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Create a new model by adding custom fully connected layers on top of ResNet-50
model_resnet50 = Sequential()
model_resnet50.add(base_model)
model_resnet50.add(GlobalAveragePooling2D())  # Use Global Average Pooling
model_resnet50.add(Dense(1024, activation='relu'))
model_resnet50.add(Dense(10, activation='softmax'))  # You can change 1000 to match your output class count

# Optionally, you can freeze the ResNet-50 layers if needed
# for layer in base_model.layers:
#     layer.trainable = False

# Compile the ResNet-50 model
model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model_resnet50.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_8 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_9 (Dense)             (None, 10)                10250     
                                                                 
Total params: 25,696,138
Trainable params: 25,643,018
Non-trainable params: 53,120
_________________________________________________________________


In [29]:
# Training
batch_size = 64
epochs = 10

# Callbacks
resnet_checkpoint = ModelCheckpoint('resnet_model.h5', save_best_only=True)
resnet_early_stop = EarlyStopping(patience=3)

# Training ResNet-50 model
model_resnet50.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                   validation_data=(x_val, y_val), callbacks=[resnet_checkpoint, resnet_early_stop])

# Evaluate the model on the test set
resnet50_test_loss, resnet50_test_accuracy = model_resnet50.evaluate(x_test, y_test)
print(f'ResNet-50 Test Loss: {resnet50_test_loss:.4f}')
print(f'ResNet-50 Test Accuracy: {resnet50_test_accuracy:.4f}')

Epoch 1/10
750/750 [==============================] - 69s 44ms/step - loss: 0.2565 - accuracy: 0.9448 - val_loss: 2.3177 - val_accuracy: 0.3714
Epoch 2/10
750/750 [==============================] - 31s 42ms/step - loss: 0.1094 - accuracy: 0.9764 - val_loss: 0.0555 - val_accuracy: 0.9854
Epoch 3/10
750/750 [==============================] - 31s 41ms/step - loss: 0.0626 - accuracy: 0.9843 - val_loss: 0.0744 - val_accuracy: 0.9809
Epoch 4/10
750/750 [==============================] - 31s 41ms/step - loss: 0.0542 - accuracy: 0.9872 - val_loss: 0.0678 - val_accuracy: 0.9825
Epoch 5/10
313/313 [==============================] - 3s 11ms/step - loss: 0.0783 - accuracy: 0.9814
ResNet-50 Test Loss: 0.0783
ResNet-50 Test Accuracy: 0.9814


In [30]:
# Define the CNN model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
model_cnn = Sequential()

model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))  # Adjust input shape
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Flatten())
model_cnn.add(Dropout(0.6))
model_cnn.add(Dense(10, activation='softmax')) 

# Compile the CNN model
model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model_cnn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 3200)             

In [32]:
# Training 
batch_size = 64
epochs = 10

# Callbacks
cnn_checkpoint = ModelCheckpoint('cnn_model.h5', save_best_only=True)
cnn_early_stop = EarlyStopping(patience=3)

# Training the CNN model
model_cnn.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
              validation_data=(x_val, y_val), callbacks=[cnn_checkpoint, cnn_early_stop])

# Evaluate the model on the test set
cnn_test_loss, cnn_test_accuracy = model_cnn.evaluate(x_test, y_test)
print(f'CNN Test Loss: {cnn_test_loss:.4f}')
print(f'CNN Test Accuracy: {cnn_test_accuracy:.4f}')

Epoch 1/10
750/750 [==============================] - 5s 5ms/step - loss: 0.2094 - accuracy: 0.9344 - val_loss: 0.0590 - val_accuracy: 0.9829
Epoch 2/10
750/750 [==============================] - 3s 4ms/step - loss: 0.0734 - accuracy: 0.9776 - val_loss: 0.0516 - val_accuracy: 0.9836
Epoch 3/10
750/750 [==============================] - 3s 4ms/step - loss: 0.0557 - accuracy: 0.9823 - val_loss: 0.0358 - val_accuracy: 0.9894
Epoch 4/10
750/750 [==============================] - 3s 4ms/step - loss: 0.0467 - accuracy: 0.9856 - val_loss: 0.0370 - val_accuracy: 0.9887
Epoch 5/10
750/750 [==============================] - 3s 5ms/step - loss: 0.0385 - accuracy: 0.9878 - val_loss: 0.0324 - val_accuracy: 0.9906
Epoch 6/10
750/750 [==============================] - 3s 4ms/step - loss: 0.0354 - accuracy: 0.9884 - val_loss: 0.0346 - val_accuracy: 0.9893
Epoch 7/10
750/750 [==============================] - 3s 4ms/step - loss: 0.0317 - accuracy: 0.9902 - val_loss: 0.0320 - val_accuracy: 0.9906
Epoch 

In [40]:
# ensembling
cnn_predictions = model_cnn.predict(x_test)
resnet50_predictions = model_resnet50.predict(x_test)

# Combine predictions using majority voting
ensemble_predictions = np.argmax(cnn_predictions + resnet50_predictions, axis=1)

ensemble_accuracy = np.mean(ensemble_predictions == np.argmax(y_test, axis=1))
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")

313/313 [==============================] - 3s 8ms/step
Ensemble Model Accuracy: 0.9926
